In [1]:
%matplotlib inline
import numpy as np
from collections import namedtuple

# Inputs

The equations below are explained in the documentation

In [18]:
LAMBDA_UM = 0.810 # Wavelength in um
N = 1.0 # Refraction index
W_10_UM = 4.0/2.0 # Beam waist radius for input beam in um
Z_1_UM = 0.0 # Axial distance from the beam's focus for input beam in um
W_00_UM = 4.0/2.0 # Beam waist radius for output beam in um
Z_0_UM = 0.0 # Axial distance from the beam's focus for input beam in um
S_UM = 0.0 # The misaligment in um
THETA_RAD = 0 # The angle misalignment in radians
PI = np.pi

# Calculate $k$, $Z_{R,1}$ and $Z_{R,0}$

In [19]:
def calculate_z_r(k, w_x0):
    return k*(w_x0**2)/2

In [20]:
k_rad_per_um = 2*PI*N/LAMBDA_UM # Wave number in radians per um
z_r1_um = calculate_z_r(k_rad_per_um, W_10_UM)
z_r0_um = calculate_z_r(k_rad_per_um, W_00_UM)

# Calculate beam parameters

In [21]:
def calculate_R(z, z_r):
    if z == 0:
        return np.inf
    else:
        return z*(1 + np.divide(z_r,z)**2)
def calculate_w(w_x0, z, z_r):
    return w_x0*np.sqrt(1 + np.divide(z, z_r)**2)

In [22]:
R_1_um = calculate_R(Z_1_UM, z_r1_um)
R_0_um = calculate_R(Z_0_UM, z_r0_um)
w_1_um = calculate_w(W_10_UM, Z_1_UM, z_r1_um)
w_0_um = calculate_w(W_00_UM, Z_0_UM, z_r0_um)
BeamParameters = namedtuple('BeamParameters', ['w', 'R'])
BParam1_um = BeamParameters(w_1_um, R_1_um)
BParam0_um = BeamParameters(w_0_um, R_0_um)

# Calculate $K$

In [23]:
K = 4.0/(
    (w_1_um*w_0_um)**2*(
        (1/w_0_um**2 + 1/w_1_um**2)**2 - 
        (k_rad_per_um*(np.divide(1, R_1_um) - np.divide(1, R_0_um))/2)**2
        )
    )

# Calculate $\eta$

In [24]:
eta = K*np.exp(
        -K*(
            S_UM**2/2*(
                1/w_1_um**2 + (k_rad_per_um*w_1_um)**2/(2*R_1_um**2) +
                1/w_0_um**2 + (k_rad_per_um*w_0_um)**2/(2*R_0_um**2)
                )
        ) +
        k_rad_per_um**2/8*(
            (THETA_RAD*w_1_um)**2 - 2*S_UM*THETA_RAD*w_1_um**2/R_1_um + 
            (THETA_RAD*w_0_um)**2 - 2*S_UM*THETA_RAD*w_0_um**2/R_0_um\
        )
    )